In [19]:
# 対象ページのURL一覧
urls = [
    'https://twitter.com',
    'https://facebook.com',
    'https://www.yahoo.co.jp'
]

from hashlib import md5
from pathlib import Path
from urllib import request

In [4]:
def download(url):
    req = request.Request(url)
    # ファイル名に/などが含まれないようにする
    name = md5(url.encode('utf-8')).hexdigest()
    file_path = './' + name
    with request.urlopen(req) as res:
        Path(file_path).write_bytes(res.read())
        return url, file_path

In [20]:
download(urls[2])

('https://www.yahoo.co.jp', './75fe82dd9f3dd946472228b906a66730')

In [12]:
import time

# 実行時間の計測を行うでコレータ
def elapsed_time(f):
    def wrapper(*args, **kwargs):
        st = time.time()
        v = f(*args, **kwargs)
        print(f'{f.__name__}: {time.time() - st}')
        return v
    return wrapper 

In [22]:
## 逐次処理で行う
@elapsed_time
def get_sequential():
    for url in urls:
        print(download(url))

get_sequential()

('https://twitter.com', './be8b09f7f1f66235a9c91986952483f0')
('https://facebook.com', './a023cfbf5f1c39bdf8407f28b60cd134')
('https://www.yahoo.co.jp', './75fe82dd9f3dd946472228b906a66730')
get_sequential: 0.9226570129394531


In [25]:
# 並行処理で行う

from concurrent.futures import (
    ThreadPoolExecutor,
    as_completed
)

@elapsed_time
def get_multi_thread():
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(download, url) for url in urls]
        for future in as_completed(futures):
            print(future.result())

In [26]:
get_multi_thread()

('https://www.yahoo.co.jp', './75fe82dd9f3dd946472228b906a66730')
('https://twitter.com', './be8b09f7f1f66235a9c91986952483f0')
('https://facebook.com', './a023cfbf5f1c39bdf8407f28b60cd134')
get_multi_thread: 0.5968329906463623


In [27]:
from concurrent.futures import (
    ThreadPoolExecutor,
    wait,
    as_completed
)

class Counter:
    def __init__(self):
        self.count = 0
    def increment(self):
        self.count = self.count + 1
        
def count_up(counter):
    for _ in range(1000000):
        counter.increment()
        
counter = Counter()
threads = 2
with ThreadPoolExecutor() as e:
    # 2つのスレッドでそれぞれcount_upを呼び出す
    futures = [e.submit(count_up, counter)
              for _ in range(threads)]
    done, not_done = wait(futures)

In [29]:
print(f'counter.count={counter.count}')

counter.count=1679396


In [30]:
import threading
class ThreadSafeCounter():
    lock = threading.Lock()
    def __init__(self):
        self.count = 0
    def increment(self):
        with self.lock:
            self.count = self.count+1

counter = ThreadSafeCounter()
threads = 2
with ThreadPoolExecutor() as e:
    futures = [e.submit(count_up, counter) 
              for _ in range(threads)]
    done, not_done = wait(futures)

In [31]:
print(f'counter.count={counter.count}')

counter.count=2000000
